### Настройка middleware

# 17-06-2025

`middleware.py`

```python
class NosniffMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        response = self.get_response(request)
        response['X-Content-Type-Options'] = 'nosniff'
        return response

import logging
from datetime import datetime

logger = logging.getLogger("analytics")

class AnalyticsMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        user_ip = request.META.get("HTTP_X_FORWARDED_FOR") or request.META.get("REMOTE_ADDR", "Неизвестный IP")
        request_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        request_path = request.path

        # Очищаем формат: без двойного времени и "IP:"
        logger.info(f"{request_time} | {user_ip} | {request_path}")

        response = self.get_response(request)
        return response

```

`Настройка settings.py`

```python
# Настройки middleware
LOGGING = {
    'version': 1,
    'disable_existing_loggers': True,
    'handlers': {
        'analytics_file': {
            'level': 'INFO',
            'class': 'logging.FileHandler',
            'filename': 'analytics.log',
            'formatter': 'detailed',
        },
    },
    'formatters': {
        'detailed': {
            'format': '%(message)s'
        }
    },
    'loggers': {
        'analytics': {
            'handlers': ['analytics_file'],
            'level': 'INFO',
            'propagate': False,
        },
    },
}
```

# 18-06-2025